LEEME antes de empezar: 

1. Apuntamos hacia el csv que queremos leer (siempre tiene que estar en la misma carpeta, en caso contrario, poner la ruta de donde se encuentra en vuestro PC)
2. De ahí ya está preparado para que nos saque el nombre de peli, el año y el ID de imdb 
3. Hace una concatenación de año y nombre para luego buscarlo en tomatoes. 
4. Empezamos con IMDB: hay Ids que no existen y primero los ignoraba, pero despues los he añadido a reusltados todo con NA para que la LEN de este resultado sea = a la LEN de tomates y podamos fusionar y hacer DF (luego lo depuramos en sql con not null)
5. Tomatitos = saca el valor de tomatometer y lo guarda en una lista 
6. Cuando tengo ambas listas las concateno, /cada tomatometer irá en lapeli que toca ya que hemos añadido tambien los NULL para asegurar el orden 
7. Hacemos el Data Frame con la columna de tomatometer añadida al final 
8. Guardamos CSV con el nombre que toque en cada caso 

¡ÁNIMO!

LO UNICO QUE HAY QUE CAMBIAR EN CADA CASO: 

- df_genero = el csv de la variable df_romance
- df_total.to_csv("romance_fase2_1990_2023.csv") el nombre del csv que guardo 


In [16]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.
import re
from pandas import DataFrame
# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import ast
import json
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.

from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.common.exceptions import NoSuchElementException

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


OJO aqui tenemos que cambiar el CSV al cual apuntemos, ahora está el "romance"

In [17]:
#EJEMPLO:cojo mi csv de pelis de romance para testear la iteración por ID en imdb 
#CAMBIAME!! 
df_genero = pd.read_csv("Thriller_10_1990_2023_copy.csv", index_col=0)
df_genero.head()

,genre,type,name,year,month,id
Unnamed: 0,,,,,,
2353,Thriller,Movie,Dark City: Interval_575,2023,9,tt11230362
2354,Thriller,Movie,Axe2Grind,2023,10,tt11995448
2355,Thriller,Movie,Karanliktaki Kadin,2023,5,tt12087678
2356,Thriller,Movie,Death Tarot,2023,2,tt12247136
2357,Thriller,Movie,65,2023,3,tt12261776


In [18]:
lista_ids = df_genero["id"]
lista_ids.head()

Unnamed: 0
2353    tt11230362
2354    tt11995448
2355    tt12087678
2356    tt12247136
2357    tt12261776
Name: id, dtype: object

In [19]:
lista_names_pelis = df_genero["name"].values
lista_años_pelis = df_genero["year"].values

In [20]:
lista_año_peli_zip = list(zip(lista_names_pelis, lista_años_pelis))
lista_año_peli_zip

[('Dark City: Interval_575', 2023),
 ('Axe2Grind', 2023),
 ('Karanliktaki Kadin', 2023),
 ('Death Tarot', 2023),
 ('65', 2023),
 ('Extraction 2', 2023),
 ('Joy', 2023),
 ('Dark Entities', 2023),
 ('72', 2023),
 ('Sharper', 2023)]

# PARTE 1 -- IMDB

In [21]:
# ABRIMOS NAVEGADOR ------------------------------------------------------

driver = webdriver.Chrome()

#Busca la web que yo quiera, en este caso IMDB
driver.get("https://www.imdb.com/")
#print("0 - Abriendo la web")

# Maximizar la ventana del navegador para mostrar la página completa
driver.maximize_window()


#------------------------- COMENZAMOS LOOP ---------------------------------

lista_imdb = []
for id in tqdm(lista_ids):
    lista_consumible = []
    
    try:

        driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER)
        sleep(4)

        lista_consumible.append(id)
        #print(" 3- Añadiendo ID")

#PUNTUACION ----------------------------------------------------------------
        try: 
            #OLD = puntuacion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-3a4309f8-0.fjtZsE.sc-dffc6c81-1.fJrHDo > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.gtEgaf > div.sc-bde20123-2.gYgHoj > span.sc-bde20123-1.iZlgcd").text
            puntuacion = driver.find_element("xpath", "//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]").text
            
            #print("4 - Sacando puntuación imdb")
            lista_consumible.append(puntuacion)
            sleep(4)
        except: 
            lista_consumible.append("NA")
            

#DIRECCION ----------------------------------------------------------------
        try: 

            # OLD director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text
            #second = director = driver.find_element("xpath", "//*[@id='__next']/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[1]/div/ul/li/a").text
            director = driver.find_element("xpath", "//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li/a").text
            
            lista_consumible.append(director)
            #print("4 - Sacando Director")
            sleep(4)
        except: 
            lista_consumible.append("NA")

#GUIONISTAS --------------------------------------------------------------
        
        try: 
            guionistas = []

            for guionista in range(1,10):
                try: 
                    #nombres_guionistas = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div
                    #nombres_guionistas = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2)
                    nombres_guionistas = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li:nth-child({guionista}) > a").text
                    #nombres_guionistas = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li:nth-child({guionista}) > a").text
                    guionistas.append(nombres_guionistas)
                except:
                    break

            lista_consumible.append(guionistas)
        except: 
            lista_consumible.append("NA")
            #print("5 - Sacando Guionistas")

#ARGUMENTO PELICULA -----------------------------------------------------
        try:

            
            argumento = driver.find_element("xpath", "/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]").text
            
        except NoSuchElementException:
            # Handle the case where the element is not found
            argumento = "Element not found"

        lista_consumible.append(argumento)

        sleep(4)

        #print("9 - Sacando Argumento")

#DURACION ---------------------------------------------------------------
        try: 
            # ANTIGUO duracion_horas = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(2)").text 
            duracion_horas = driver.find_element("xpath", "//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]").text
            #duracion_horas = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-dffc6c81-0.grcyBP > ul > li:nth-child(3)").text 
            # #print("6 - Sacando duración")
            #PARA CONVERTIR LA DURACION TOTAL A MINUTOS! 
            partes = duracion_horas.split() 

            horas = 0 
            minutos = 0 

            for parte in partes:
                if "h" in parte: 
                    horas = horas + int(parte[:-1]) 
                elif "m" in parte: 
                    minutos = minutos + int(parte[:-1])# :-1 al eliminar el H y el M entonces nos da el digit. Si pongo [0]me daría solo el 2 de 22, o el 2 de 25.

            duracion = horas * 60 + minutos 

            lista_consumible.append(duracion)
            sleep(6)
            #print("7 - convirtiendo a minutos")
        except: 
            lista_consumible.append("NA")

#NOMBRE PELICULA ----------------------------------------------------------------
        try: 
            nombre_imdb = driver.find_element("xpath", "//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1/span").text
            #nombre_imdb = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-dffc6c81-0.grcyBP > h1 > span").text
            #nombre_imdb = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-dffc6c81-0.grcyBP > h1 > span").text
            # ANTIGUO = nombre_imdb = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > h1 > span").text
            sleep(4)
            lista_consumible.append(nombre_imdb)
            
            #print("8 - Sacando nombre Peli")
            sleep(4)
        except:
            lista_consumible.append("NA")

#----------------------------------------------------------------
# He añadido este parche para que me añada los casos de Ids de pelis que ya no existen en imdb pero que los busco igualmente en rotten tomatoes. 
# Luego se puede limpiar en BBDD pero así la len de ambas listas es = para hacer el dataframe y guardar el CSV 

    except NoSuchElementException:
        # Cuando el ID que estamos buscando no existe en imdb
        print(f"ID {id} does not exist on IMDb. Skipping...")
        puntuacion = "NA"
        lista_consumible.append(puntuacion)
        nombre_imdb = "NA"
        lista_consumible.append(nombre_imdb)
        duracion = "NA"
        lista_consumible.append(duracion)
        director = "NA"
        lista_consumible.append(director)
        argumento = "NA"
        lista_consumible.append(argumento)
        guionistas = "NA"
        lista_consumible.append(guionistas)

        driver.back()
        sleep(8)



# FUERA DEL LOOP: 

    if lista_consumible: #evita hacer append de tuplas vacías
                            
        lista_imdb.append(lista_consumible)
        #FALTA CONVERTIRLO A TUPLA


 70%|███████   | 7/10 [03:56<01:36, 32.02s/it]

ID tt12476674 does not exist on IMDb. Skipping...


100%|██████████| 10/10 [05:52<00:00, 35.20s/it]


In [23]:
df_imdb = pd.DataFrame(lista_imdb, columns=["ID", "Puntuación", "Dirección", "Guionistas", "Argumento", "Duración", "Nombre imdb"])
df_imdb

,ID,Puntuación,Dirección,Guionistas,Argumento,Duración,Nombre imdb
0,tt11230362,NA,Cecil Moller,[Gustav Nuuyoma],"Toivo, a university student comes back to the ...",NA,Dark City: Interval_575
1,tt11995448,NA,Stormy Daniels,"[Stormy Daniels, Spencer Gray, Matt Gulbranson]",Debbie Wilkins is released after serving 10 ye...,NA,Axe2Grind
2,tt12087678,NA,Aksel Yakup Bektas,[Aksel Yakup Bektas],The scary story of a woman who died nearly 100...,NA,Karanliktaki Kadin
3,tt12247136,NA,Joe Hammerstone,"[Joe Hammerstone, Casey Lee McIntosh]",Hayley and Ken try to solve the murders of loc...,NA,Death Tarot
4,tt12261776,5.4,Scott Beck,"[Scott Beck, Bryan Woods]",An astronaut crash lands on a mysterious plane...,93,65
5,tt12263384,7.0,Sam Hargrave,"[Joe Russo, Ande Parks, Anthony Russo]",After barely surviving his grievous wounds fro...,122,Extraction 2
6,tt12370358,NA,NA,[],Element not found,NA,NA
7,NA,NA,NA,NA,NA,NA,None
8,tt12494932,NA,Carl Miller,[Carl Miller],A highly fatal and contagious virus that hits ...,NA,72
9,tt12573454,6.7,Benjamin Caron,"[Brian Gatewood, Alessandro Tanaka]","Motivations are suspect, and expectations are ...",116,Sharper


In [ ]:
df_imdb.to_csv("Thriller_copy_test_sintomate.csv")

# PARTE 2 - TOMATITOS 

In [ ]:

# Website 2 - Rotten Tomatoes

# Inicializa el navegador de Chrome--------------------------
driver = webdriver.Chrome()

# Abre Google en el navegador--------------------------
driver.get("https://www.google.com/")
#print("1 - Abriendo Google")

# Maximiza la ventana del navegador--------------------------
driver.maximize_window()

# Acepta las cookies de Google--------------------------

try:
    driver.find_element(By.CSS_SELECTOR, "#L2AGLb > div").click()
    #print("2 - Cookies de Google")
except NoSuchElementException:
    print("El botón de cookies no se encontró, continuando con el código.")


lista_tomates = []

# Bucle para buscar las películas --------------------------
for año, título in tqdm(lista_año_peli_zip):

# Realiza una búsqueda en Google --------------------------
    driver.get("https://www.google.com/")
    #print("3 - Entrando a Google"

    busqueda_pelicula = f"{año} {título} Rotten Tomatoes"
    # driver.find_element("css selector", "#APjFqb").clear()
    driver.find_element(By.CSS_SELECTOR, "#APjFqb").send_keys(
        busqueda_pelicula, Keys.ENTER
    )

    try:
        element = driver.find_element(
            By.XPATH,
            "/html/body/div[5]/div/div[12]/div/div[1]/div[1]/div/div/div[3]/div/div/div[2]/div/g-raised-button[1]/div/span",
        )

    except NoSuchElementException:
        print("Sign up de google passed.")
        # SI ENTRO AQUI, IRÉ DIRECTAMENTE EL SIGUIENTE TRY (ES COMO UN PASS)
    else:
        element.click()

# Haz clic en el primer resultado de búsqueda ----------------------
    try:
        element = driver.find_element(By.XPATH, "//*[@id='rso']/div[1]/descendant::h3")

    except NoSuchElementException:
        print(
            "No se encontró el elemento. Puede ser necesario actualizar el selector CSS."
        )

        continue

    element.click()

    #print("5 - Haciendo clic en la búsqueda")

# Acepta las cookies en la página de Rotten Tomatoes --------------------------

    try:
        element = driver.find_element(By.CSS_SELECTOR, "#onetrust-accept-btn-handler")

    except NoSuchElementException:
        print("Cookies de rotten")
    # SI ENTRO AQUI, IRÉ DIRECTAMENTE EL SIGUIENTE TRY (ES COMO UN PASS)
    else:
        element.click()

    #print("6 - Aceptando cookies en Rotten Tomatoes")

    try:
# Accede a la URL de Rotten Tomatoes para obtener información --------------------------

        tomatometer_element = driver.find_element("xpath", "//*[@id='scoreboard']")
        tomatometer = tomatometer_element.get_attribute("tomatometerscore")

        if tomatometer:
            lista_tomates.append([f"{tomatometer}%"])
            #print(f"Tomatometer Value for {año} - {título}: {tomatometer}")
        else:
            lista_tomates.append(["--"])
 
            #print("Tomatometer not found")
    except NoSuchElementException:
        lista_tomates.append(["--"])
    
        #print("No se encontró el tomato")

driver.quit()



In [ ]:
df_imdb = pd.DataFrame(lista_imdb, columns=["ID", "Puntuación", "Dirección", "Guionistas", "Argumento", "Duración", "Nombre imdb"])
df_imdb

In [ ]:
len(lista_imdb)

In [ ]:
len(lista_tomates)

In [ ]:
df_tomates = pd.DataFrame(lista_tomates, columns=["tomatometer"])
df_tomates


FUSIONAMOS AMBAS LISTAS DE RESULTADOS 

In [ ]:
lista_imdb 
lista_tomates 
listas_imdb_tomate = []

for imdb, tomates in zip(lista_imdb, lista_tomates):
    lista_combinada = imdb + tomates
    listas_imdb_tomate.append(tuple(lista_combinada))

print(listas_imdb_tomate)

In [ ]:

listas_imdb_tomate


YA TENEMOS TOMATOMETER AÑADIDO 

In [ ]:
df_total = pd.DataFrame(listas_imdb_tomate, columns=["ID", "Puntuación", "Dirección", "Guionistas", "Argumento", "Duración", "Nombre imdb", "tomatometer"])
df_total

Guardar todo lo que hagamos con nombre de genero + fase + año inicio + año fin

#vamos cambiando el nombre del csv por cada tramo de pelis que hagamos

In [ ]:
df_total.to_csv("Thriller_Fase2_10_1990_2023_CON_tomate.csv")